In [1]:
import ipycytoscape
import covidkg as cvkg
import pandas as pd
from covidkg import Node, Edge, KnowledgeGraph

In [2]:
nodes_data = pd.read_csv("data/nodes_data.csv")
interactions_data = pd.read_csv("data/interactions_data.csv")
tax_id = nodes_data["TaxonomyID"].astype(str)
href = "https://www.uniprot.org/uniprot/" + nodes_data["Identifier"].str[8:14]

In [3]:
kg = KnowledgeGraph()

def create_nodes(n_name, n_id, kg):
    for i in range(len(n_name)):
        kg.add_node(Node(n_name[i], n_id[i], tax_id[i], href[i]))

def create_edges(source, target, kg):
    for i in range(len(source)):
        kg.add_edge(Edge(Node("source_" + str(i), source[i]), Node("target_" + str(i), target[i])))
        
create_nodes(nodes_data["Name"], nodes_data["ID"], kg)

create_edges(interactions_data["SARS_COV2_Protein_ID"].values, interactions_data["Human_Protein_ID"].values, kg)

In [4]:
kg.load_graph()

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'css': {'content': 'd…